# 📗 JavaScript Advanced Concepts
Deep dive into closures, prototypes, async programming, and ES6+ features.

---
## 1. Scope & Closures

### Concept: Scope Chain
JavaScript has 3 types of scope:
- **Global Scope**: Variables accessible everywhere
- **Function Scope**: Variables inside a function
- **Block Scope**: Variables inside `{}` (let/const only)

In [ ]:
// Scope Chain Example
let global = "I'm global";

function outer() {
    let outerVar = "I'm from outer";
    
    function inner() {
        let innerVar = "I'm from inner";
        console.log(innerVar);  // Own scope
        console.log(outerVar);  // Parent scope
        console.log(global);    // Global scope
    }
    
    inner();
}

outer();

### Concept: Closures
A closure is a function that remembers its outer variables and can access them.

In [ ]:
// Basic Closure
function createCounter() {
    let count = 0;  // Private variable
    
    return function() {
        count++;
        return count;
    };
}

const counter = createCounter();
console.log(counter()); // 1
console.log(counter()); // 2
console.log(counter()); // 3

const counter2 = createCounter(); // New closure
console.log(counter2()); // 1

### ❓ Q1: Create a function that returns functions for add, subtract, multiply

In [ ]:
function calculator(initialValue = 0) {
    let value = initialValue;
    
    return {
        add(n) { value += n; return this; },
        subtract(n) { value -= n; return this; },
        multiply(n) { value *= n; return this; },
        divide(n) { value /= n; return this; },
        getValue() { return value; }
    };
}

const calc = calculator(10);
console.log(calc.add(5).multiply(2).subtract(10).getValue()); // 20

### ❓ Q2: Classic closure interview question

In [ ]:
// Problem: What does this print?
for (var i = 0; i < 3; i++) {
    setTimeout(() => console.log("var:", i), 100);
}
// Output: 3, 3, 3 (var is function-scoped, same reference)

// Solution 1: Use let (block-scoped)
for (let j = 0; j < 3; j++) {
    setTimeout(() => console.log("let:", j), 200);
}
// Output: 0, 1, 2

// Solution 2: Use IIFE (Immediately Invoked Function Expression)
for (var k = 0; k < 3; k++) {
    ((k) => {
        setTimeout(() => console.log("IIFE:", k), 300);
    })(k);
}
// Output: 0, 1, 2

### ❓ Q3: Implement memoization using closures

In [ ]:
function memoize(fn) {
    const cache = {};
    
    return function(...args) {
        const key = JSON.stringify(args);
        
        if (cache[key] !== undefined) {
            console.log('From cache');
            return cache[key];
        }
        
        console.log('Computing');
        const result = fn.apply(this, args);
        cache[key] = result;
        return result;
    };
}

const expensiveAdd = memoize((a, b) => a + b);

console.log(expensiveAdd(1, 2)); // Computing -> 3
console.log(expensiveAdd(1, 2)); // From cache -> 3
console.log(expensiveAdd(3, 4)); // Computing -> 7

---
## 2. The `this` Keyword

In [ ]:
// 'this' depends on HOW the function is called

// 1. Global context
console.log(this); // window (browser) or global (Node)

// 2. Object method
const obj = {
    name: "Object",
    greet() {
        console.log(this.name); // "Object"
    }
};
obj.greet();

// 3. Arrow function (inherits from parent)
const obj2 = {
    name: "Object2",
    greet: () => {
        console.log(this.name); // undefined (inherits global this)
    }
};
obj2.greet();

In [ ]:
// call, apply, bind

function greet(greeting, punctuation) {
    console.log(`${greeting}, ${this.name}${punctuation}`);
}

const person = { name: "Alice" };

// call - invoke immediately with args
greet.call(person, "Hello", "!"); // "Hello, Alice!"

// apply - invoke immediately with args array
greet.apply(person, ["Hi", "?"]); // "Hi, Alice?"

// bind - return new function with bound this
const boundGreet = greet.bind(person);
boundGreet("Hey", "."); // "Hey, Alice."

### ❓ Q4: Implement your own bind, call, apply

In [ ]:
// Custom call
Function.prototype.myCall = function(context, ...args) {
    context = context || globalThis;
    const sym = Symbol();
    context[sym] = this;
    const result = context[sym](...args);
    delete context[sym];
    return result;
};

// Custom apply
Function.prototype.myApply = function(context, args = []) {
    return this.myCall(context, ...args);
};

// Custom bind
Function.prototype.myBind = function(context, ...args) {
    const fn = this;
    return function(...newArgs) {
        return fn.myCall(context, ...args, ...newArgs);
    };
};

// Test
function sayHi(msg) { return `${msg}, ${this.name}`; }
const user = { name: "Bob" };

console.log(sayHi.myCall(user, "Hello"));  // "Hello, Bob"
console.log(sayHi.myApply(user, ["Hi"])); // "Hi, Bob"
console.log(sayHi.myBind(user)("Hey"));   // "Hey, Bob"

---
## 3. Prototypes & Inheritance

In [ ]:
// Every object has a prototype
const arr = [1, 2, 3];
console.log(arr.__proto__ === Array.prototype); // true
console.log(Array.prototype.__proto__ === Object.prototype); // true
console.log(Object.prototype.__proto__); // null (end of chain)

// Constructor Function
function Person(name, age) {
    this.name = name;
    this.age = age;
}

Person.prototype.greet = function() {
    return `Hi, I'm ${this.name}`;
};

const john = new Person("John", 30);
console.log(john.greet()); // "Hi, I'm John"
console.log(john.__proto__ === Person.prototype); // true

In [ ]:
// ES6 Classes (syntactic sugar over prototypes)
class Animal {
    constructor(name) {
        this.name = name;
    }
    
    speak() {
        return `${this.name} makes a sound`;
    }
}

class Dog extends Animal {
    constructor(name, breed) {
        super(name);  // Call parent constructor
        this.breed = breed;
    }
    
    speak() {
        return `${this.name} barks!`;
    }
}

const dog = new Dog("Rex", "German Shepherd");
console.log(dog.speak()); // "Rex barks!"
console.log(dog instanceof Dog);    // true
console.log(dog instanceof Animal); // true

### ❓ Q5: Implement classical inheritance without ES6 classes

In [ ]:
// Parent
function Vehicle(make, model) {
    this.make = make;
    this.model = model;
}

Vehicle.prototype.getInfo = function() {
    return `${this.make} ${this.model}`;
};

// Child
function Car(make, model, doors) {
    Vehicle.call(this, make, model);  // Call parent constructor
    this.doors = doors;
}

// Set up inheritance
Car.prototype = Object.create(Vehicle.prototype);
Car.prototype.constructor = Car;

// Add child method
Car.prototype.honk = function() {
    return "Beep beep!";
};

const myCar = new Car("Toyota", "Camry", 4);
console.log(myCar.getInfo()); // "Toyota Camry"
console.log(myCar.honk());    // "Beep beep!"
console.log(myCar instanceof Car);     // true
console.log(myCar instanceof Vehicle); // true

---
## 4. Promises & Async/Await

In [ ]:
// Creating a Promise
const myPromise = new Promise((resolve, reject) => {
    setTimeout(() => {
        const success = true;
        if (success) {
            resolve("Operation succeeded!");
        } else {
            reject(new Error("Operation failed!"));
        }
    }, 1000);
});

myPromise
    .then(result => console.log(result))
    .catch(error => console.error(error))
    .finally(() => console.log("Done"));

In [ ]:
// Promise chaining
function fetchUser(id) {
    return new Promise(resolve => {
        setTimeout(() => resolve({ id, name: "User" + id }), 100);
    });
}

function fetchPosts(userId) {
    return new Promise(resolve => {
        setTimeout(() => resolve(["Post1", "Post2"]), 100);
    });
}

fetchUser(1)
    .then(user => {
        console.log(user);
        return fetchPosts(user.id);
    })
    .then(posts => console.log(posts))
    .catch(err => console.error(err));

In [ ]:
// Async/Await (cleaner syntax)
async function getUserWithPosts(id) {
    try {
        const user = await fetchUser(id);
        console.log(user);
        const posts = await fetchPosts(user.id);
        console.log(posts);
        return { user, posts };
    } catch (error) {
        console.error("Error:", error);
    }
}

getUserWithPosts(1);

In [ ]:
// Promise static methods
const p1 = Promise.resolve(1);
const p2 = Promise.resolve(2);
const p3 = Promise.resolve(3);

// Promise.all - wait for all (fails fast)
Promise.all([p1, p2, p3]).then(values => console.log("all:", values)); // [1, 2, 3]

// Promise.allSettled - wait for all (doesn't fail)
Promise.allSettled([p1, Promise.reject("err"), p3])
    .then(results => console.log("allSettled:", results));

// Promise.race - first to settle
Promise.race([p1, p2, p3]).then(value => console.log("race:", value)); // 1

// Promise.any - first to fulfill
Promise.any([Promise.reject("err"), p2, p3])
    .then(value => console.log("any:", value)); // 2

### ❓ Q6: Implement Promise.all from scratch

In [ ]:
function myPromiseAll(promises) {
    return new Promise((resolve, reject) => {
        const results = [];
        let completed = 0;
        
        if (promises.length === 0) {
            resolve(results);
            return;
        }
        
        promises.forEach((promise, index) => {
            Promise.resolve(promise)
                .then(value => {
                    results[index] = value;
                    completed++;
                    
                    if (completed === promises.length) {
                        resolve(results);
                    }
                })
                .catch(reject);
        });
    });
}

// Test
myPromiseAll([
    Promise.resolve(1),
    Promise.resolve(2),
    Promise.resolve(3)
]).then(console.log); // [1, 2, 3]

### ❓ Q7: Implement a retry mechanism

In [ ]:
async function retry(fn, maxRetries = 3, delay = 1000) {
    let lastError;
    
    for (let i = 0; i < maxRetries; i++) {
        try {
            return await fn();
        } catch (error) {
            lastError = error;
            console.log(`Attempt ${i + 1} failed. Retrying...`);
            
            if (i < maxRetries - 1) {
                await new Promise(r => setTimeout(r, delay));
            }
        }
    }
    
    throw lastError;
}

// Usage
let attempts = 0;
const flakeyFunction = () => {
    attempts++;
    if (attempts < 3) throw new Error("Failed");
    return "Success!";
};

retry(flakeyFunction, 5, 100)
    .then(console.log)
    .catch(console.error);

---
## 5. Event Loop & Execution Order

In [ ]:
// Understanding the event loop
console.log("1: Start");

setTimeout(() => console.log("2: Timeout 0"), 0);

Promise.resolve().then(() => console.log("3: Promise 1"));

Promise.resolve().then(() => {
    console.log("4: Promise 2");
    Promise.resolve().then(() => console.log("5: Nested Promise"));
});

console.log("6: End");

// Output order: 1, 6, 3, 4, 5, 2
// Because: Sync code first, then microtasks (Promises), then macrotasks (setTimeout)

### ❓ Q8: Predict the output

In [ ]:
async function async1() {
    console.log("async1 start");
    await async2();
    console.log("async1 end");
}

async function async2() {
    console.log("async2");
}

console.log("script start");

setTimeout(() => console.log("setTimeout"), 0);

async1();

new Promise(resolve => {
    console.log("promise1");
    resolve();
}).then(() => console.log("promise2"));

console.log("script end");

/*
Output:
1. script start
2. async1 start
3. async2
4. promise1
5. script end
6. async1 end
7. promise2
8. setTimeout
*/

---
## 6. Higher-Order Functions

In [ ]:
// Function that returns a function
function multiply(factor) {
    return function(number) {
        return number * factor;
    };
}

const double = multiply(2);
const triple = multiply(3);

console.log(double(5)); // 10
console.log(triple(5)); // 15

// Function composition
const compose = (...fns) => (x) => fns.reduceRight((acc, fn) => fn(acc), x);
const pipe = (...fns) => (x) => fns.reduce((acc, fn) => fn(acc), x);

const add10 = x => x + 10;
const mult2 = x => x * 2;

const composedFn = compose(add10, mult2); // mult2 first, then add10
console.log(composedFn(5)); // (5 * 2) + 10 = 20

const pipedFn = pipe(add10, mult2); // add10 first, then mult2
console.log(pipedFn(5)); // (5 + 10) * 2 = 30

### ❓ Q9: Implement curry function

In [ ]:
function curry(fn) {
    return function curried(...args) {
        if (args.length >= fn.length) {
            return fn.apply(this, args);
        }
        return function(...moreArgs) {
            return curried.apply(this, args.concat(moreArgs));
        };
    };
}

function add(a, b, c) {
    return a + b + c;
}

const curriedAdd = curry(add);

console.log(curriedAdd(1)(2)(3));   // 6
console.log(curriedAdd(1, 2)(3));   // 6
console.log(curriedAdd(1)(2, 3));   // 6
console.log(curriedAdd(1, 2, 3));   // 6

### ❓ Q10: Implement debounce and throttle

In [ ]:
// Debounce: Wait until user stops triggering
function debounce(fn, delay) {
    let timeoutId;
    
    return function(...args) {
        clearTimeout(timeoutId);
        timeoutId = setTimeout(() => {
            fn.apply(this, args);
        }, delay);
    };
}

// Throttle: Execute at most once per interval
function throttle(fn, limit) {
    let inThrottle;
    
    return function(...args) {
        if (!inThrottle) {
            fn.apply(this, args);
            inThrottle = true;
            setTimeout(() => inThrottle = false, limit);
        }
    };
}

// Usage
const debouncedLog = debounce(console.log, 300);
const throttledLog = throttle(console.log, 300);

// Debounce: Only last call executes after 300ms of inactivity
// Throttle: Executes first, then ignores for 300ms

---
## 7. Modules (ES6)

In [ ]:
// ES6 Module syntax (example - cannot run in notebook directly)

// math.js - Named exports
// export const add = (a, b) => a + b;
// export const subtract = (a, b) => a - b;
// export default function multiply(a, b) { return a * b; }

// main.js - Imports
// import multiply, { add, subtract } from './math.js';
// import * as math from './math.js';

// Dynamic import
// const module = await import('./math.js');

console.log("Module syntax examples (see code comments)");

---
## 8. Proxy & Reflect

In [ ]:
// Proxy: Intercept operations on objects
const target = { name: "John", age: 30 };

const handler = {
    get(target, prop) {
        console.log(`Getting ${prop}`);
        return prop in target ? target[prop] : "Property not found";
    },
    set(target, prop, value) {
        console.log(`Setting ${prop} to ${value}`);
        if (prop === 'age' && typeof value !== 'number') {
            throw new TypeError('Age must be a number');
        }
        target[prop] = value;
        return true;
    }
};

const proxy = new Proxy(target, handler);

console.log(proxy.name);    // Getting name -> John
proxy.age = 31;             // Setting age to 31
console.log(proxy.unknown); // Getting unknown -> Property not found

### ❓ Q11: Create a reactive object using Proxy

In [ ]:
function reactive(obj, onChange) {
    return new Proxy(obj, {
        set(target, prop, value) {
            const oldValue = target[prop];
            target[prop] = value;
            onChange(prop, value, oldValue);
            return true;
        }
    });
}

const state = reactive({ count: 0 }, (prop, newVal, oldVal) => {
    console.log(`${prop} changed from ${oldVal} to ${newVal}`);
});

state.count = 1; // count changed from 0 to 1
state.count = 2; // count changed from 1 to 2

---
## 9. Generators & Iterators

In [ ]:
// Generator function
function* numberGenerator() {
    yield 1;
    yield 2;
    yield 3;
}

const gen = numberGenerator();
console.log(gen.next()); // { value: 1, done: false }
console.log(gen.next()); // { value: 2, done: false }
console.log(gen.next()); // { value: 3, done: false }
console.log(gen.next()); // { value: undefined, done: true }

// Infinite sequence
function* infiniteSequence() {
    let i = 0;
    while (true) {
        yield i++;
    }
}

const inf = infiniteSequence();
console.log(inf.next().value); // 0
console.log(inf.next().value); // 1

### ❓ Q12: Create a custom iterable

In [ ]:
const range = {
    from: 1,
    to: 5,
    
    [Symbol.iterator]() {
        let current = this.from;
        const last = this.to;
        
        return {
            next() {
                if (current <= last) {
                    return { value: current++, done: false };
                }
                return { done: true };
            }
        };
    }
};

for (const num of range) {
    console.log(num); // 1, 2, 3, 4, 5
}

console.log([...range]); // [1, 2, 3, 4, 5]

---
## 10. Advanced Practice Problems

### Problem 1: Event Emitter

In [ ]:
class EventEmitter {
    constructor() {
        this.events = {};
    }
    
    on(event, listener) {
        if (!this.events[event]) {
            this.events[event] = [];
        }
        this.events[event].push(listener);
        return this;
    }
    
    off(event, listener) {
        if (this.events[event]) {
            this.events[event] = this.events[event]
                .filter(l => l !== listener);
        }
        return this;
    }
    
    emit(event, ...args) {
        if (this.events[event]) {
            this.events[event].forEach(listener => listener(...args));
        }
        return this;
    }
    
    once(event, listener) {
        const wrapper = (...args) => {
            listener(...args);
            this.off(event, wrapper);
        };
        return this.on(event, wrapper);
    }
}

const emitter = new EventEmitter();
emitter.on('greet', name => console.log(`Hello, ${name}!`));
emitter.emit('greet', 'World'); // Hello, World!

### Problem 2: Deep Equality Check

In [ ]:
function deepEqual(a, b) {
    // Same reference or same primitive
    if (a === b) return true;
    
    // Null checks
    if (a === null || b === null) return false;
    
    // Type check
    if (typeof a !== typeof b) return false;
    
    // Not objects - already checked equality above
    if (typeof a !== 'object') return false;
    
    // Array check
    if (Array.isArray(a) !== Array.isArray(b)) return false;
    
    // Compare keys
    const keysA = Object.keys(a);
    const keysB = Object.keys(b);
    
    if (keysA.length !== keysB.length) return false;
    
    // Deep compare values
    return keysA.every(key => deepEqual(a[key], b[key]));
}

console.log(deepEqual({a: {b: 1}}, {a: {b: 1}})); // true
console.log(deepEqual({a: {b: 1}}, {a: {b: 2}})); // false
console.log(deepEqual([1, [2, 3]], [1, [2, 3]])); // true

### Problem 3: Promise Pool (Concurrent Limit)

In [ ]:
async function promisePool(tasks, concurrency) {
    const results = [];
    let index = 0;
    
    async function runNext() {
        while (index < tasks.length) {
            const currentIndex = index++;
            results[currentIndex] = await tasks[currentIndex]();
        }
    }
    
    const workers = [];
    for (let i = 0; i < Math.min(concurrency, tasks.length); i++) {
        workers.push(runNext());
    }
    
    await Promise.all(workers);
    return results;
}

// Example usage
const createTask = (id, delay) => () => 
    new Promise(r => setTimeout(() => {
        console.log(`Task ${id} complete`);
        r(id);
    }, delay));

const tasks = [
    createTask(1, 100),
    createTask(2, 50),
    createTask(3, 150),
    createTask(4, 80)
];

promisePool(tasks, 2).then(console.log); // [1, 2, 3, 4]

---
## 🎯 Summary

This notebook covered:
- Closures and Scope
- The `this` keyword and context
- Prototypes and Inheritance
- Promises and Async/Await
- Event Loop
- Higher-Order Functions
- Proxy and Reflect
- Generators and Iterators

**Next**: Data Structures and Algorithms